# Preparation stuff

## Connect to Drive

In [1]:
connect_to_drive = False

In [2]:
#Run command and authorize by popup --> other window
if connect_to_drive:
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)

## Install packages

In [3]:
if connect_to_drive:
    #Install FS code
    !pip install  --upgrade --force-reinstall git+https://github.com/federicosiciliano/easy_lightning.git

    !pip install pytorch_lightning

## IMPORTS

In [4]:
#Put all imports here
import numpy as np
from copy import deepcopy
import os
import sys

from ray import tune
from ray.train.torch import TorchTrainer
from ray.train import ScalingConfig#, RunConfig, CheckpointConfig

## Define paths

In [5]:
#every path should start from the project folder:
project_folder = "../"
if connect_to_drive:
    project_folder = "/content/gdrive/Shareddrives/<SharedDriveName>" #Name of SharedDrive folder
    #project_folder = "/content/gdrive/MyDrive/<MyDriveName>" #Name of MyDrive folder

#Config folder should contain hyperparameters configurations
cfg_folder = os.path.join(project_folder,"cfg")

#Data folder should contain raw and preprocessed data
data_folder = os.path.join(project_folder,"data")
raw_data_folder = os.path.join(data_folder,"raw")
processed_data_folder = os.path.join(data_folder,"processed")

#Source folder should contain all the (essential) source code
source_folder = os.path.join(project_folder,"src")

#The out folder should contain all outputs: models, results, plots, etc.
out_folder = os.path.join(project_folder,"out")
img_folder = os.path.join(out_folder,"img")

## Import own code

In [6]:
#To import from src:

#attach the source folder to the start of sys.path
sys.path.insert(0, project_folder)

#import from src directory
# from src import ??? as additional_module
import easy_rec as additional_module #REMOVE THIS LINE IF IMPORTING OWN ADDITIONAL MODULE

import easy_exp, easy_rec, easy_torch #easy_data

# MAIN

## Train

### Data

In [7]:
cfg = easy_exp.cfg.load_configuration("config_rec")

In [9]:
def prepare_raytune_config(cfg):
    raytune_cfg = {}
    for parameter_name, v in cfg["__exp__"]["__sweep__"]["parameters"].items():
        if "tune" in v:
            raytune_cfg[parameter_name] = getattr(tune, v["tune"]["name"])(**v["tune"]["params"])
    return raytune_cfg

In [10]:
raytune_cfg = prepare_raytune_config(cfg)

In [11]:
#from ray.train.lightning import RayDDPStrategy, RayLightningEnvironment, RayTrainReportCallback, prepare_trainer

In [12]:
# cfg["model"]["trainer_params"]["strategy"] = RayDDPStrategy()
# #cfg["model"]["trainer_params"]["callbacks"].append(lambda : RayTrainReportCallback())
# cfg["model"]["trainer_params"]["plugins"] = cfg["model"]["trainer_params"].get("plugins", []) + [RayLightningEnvironment()]

In [13]:
def run_config(cfg, if_exp_found=None, raytune=False):
    # exp_found
    # skip --> skip the experiment
    # load --> load the experiment
    # if not load nor skip, reruns the experiment completely

    exp_found, experiment_id = easy_exp.exp.get_set_experiment_id(cfg)
    #print("Experiment already found:", exp_found, "----> The experiment id is:", experiment_id)

    if exp_found and if_exp_found == "skip":
        #print("Skipping experiment")
        return
    
    # Save experiment (done here cause Early stopping with Tune schedulers may not run anything after training)
    easy_exp.exp.save_experiment(cfg)

    data, maps = easy_rec.preparation.prepare_rec_data(cfg)

    loaders = easy_rec.preparation.prepare_rec_dataloaders(cfg, data, maps)

    main_module = easy_rec.preparation.prepare_rec_model(cfg, maps)

    trainer = easy_torch.preparation.complete_prepare_trainer(cfg, experiment_id, additional_module=easy_rec, raytune=raytune)

    model = easy_torch.preparation.complete_prepare_model(cfg, main_module, additional_module=easy_rec)

    if exp_found and if_exp_found == "load":
        easy_torch.process.load_model(trainer, model, experiment_id)

    easy_torch.process.test_model(trainer, model, loaders, test_key=["val","test","train"])

    # Train the model using the prepared trainer, model, and data loaders
    easy_torch.process.train_model(trainer, model, loaders, val_key=["val","test"])

    # Early stopping with Tune schedulers may not run anything after training

In [14]:
def run_raytune_cfg(raytune_cfg, cfg, if_exp_found=None):
    complete_cfg = deepcopy(cfg)
    complete_cfg.update(raytune_cfg)

    # save complete_cfg to a file

    run_config(complete_cfg, if_exp_found, raytune=True)

In [15]:
# checkpoint_data = {
#     "epoch": epoch,
#     "net_state_dict": net.state_dict(),
#     "optimizer_state_dict": optimizer.state_dict(),
# }
# with tempfile.TemporaryDirectory() as checkpoint_dir:
#     data_path = Path(checkpoint_dir) / "data.pkl"
#     with open(data_path, "wb") as fp:
#         pickle.dump(checkpoint_data, fp)

#     checkpoint = Checkpoint.from_directory(checkpoint_dir)
#     train.report(
#         {"loss": val_loss / val_steps, "accuracy": correct / total},
#         checkpoint=checkpoint,
#     )

In [16]:
max_num_epochs = cfg["model"]["trainer_params"]["max_epochs"]
scheduler = tune.schedulers.ASHAScheduler(
        metric="val_NDCG_@10/dataloader_idx_0",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)

In [17]:
scaling_config = ScalingConfig(
    num_workers=2, use_gpu=True, resources_per_worker={"CPU": 4, "GPU": 1}
)

# run_config = RunConfig(
#     checkpoint_config=CheckpointConfig(
#         num_to_keep=2,
#         checkpoint_score_attribute="ptl/val_accuracy",
#         checkpoint_score_order="max",
#     ),
# )

# Define a TorchTrainer without hyper-parameters for Tuner
ray_trainer = TorchTrainer(
    lambda x: run_raytune_cfg(x, cfg),
    # scaling_config=scaling_config,
    # run_config=run_config,
)



In [18]:
os.environ["RAY_CHDIR_TO_TRIAL_DIR"] = "0" #To avoid changing working directory

In [19]:
tuner = tune.Tuner(
    ray_trainer,
    param_space={"train_loop_config": raytune_cfg},
    tune_config=tune.TuneConfig(
        # metric="val_NDCG_@10/dataloader_idx_0",
        # mode="max",
        num_samples=100,
        scheduler=scheduler,
        time_budget_s=40, #seconds #May raise WARNING Failed to fetch metrics for
        max_concurrent_trials=2,
    ),
)

results = tuner.fit()

(TorchTrainer pid=2997) Started distributed worker processes: 
(TorchTrainer pid=2997) - (node_id=fed15232356329568052a5376b724ec8d5728378a80c03a3ab3695fd, ip=127.0.0.1, pid=3011) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=3011) Setting up process group for: env:// [rank=0, world_size=1]


(RayTrainWorker pid=3012) Ratings data already exists. Skip pre-processing
(RayTrainWorker pid=3014) ../out/exp/prova not found --> created
(RayTrainWorker pid=3011) ../out/exp not found --> created
(RayTrainWorker pid=3012) Filtering by minimum number of users per item: 5
(RayTrainWorker pid=3012) Filtering by minimum number of items per user: 5
(RayTrainWorker pid=3012) Densifying index
(RayTrainWorker pid=3012) Splitting: leave_n_out
(RayTrainWorker pid=3012) Warning: RayTrainReportCallback not recognized for adding experiment_id


(RayTrainWorker pid=3011) [rank: 0] Seed set to 42
(RayTrainWorker pid=3011) [rank: 0] Seed set to 42
(RayTrainWorker pid=3011) [rank: 0] Seed set to 42
(RayTrainWorker pid=3011) [rank: 0] Seed set to 42
(RayTrainWorker pid=3011) GPU available: True (mps), used: False
(RayTrainWorker pid=3011) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=3011) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=3011) /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
(RayTrainWorker pid=3011) [rank: 0] Seed set to 42
(RayTrainWorker pid=3011) [rank: 0] Seed set to 42
(RayTrainWorker pid=3011) [rank: 0] Seed set to 42
(RayTrainWorker pid=3011) [rank: 0] Seed set to 42
(RayTrainWorker pid=3011) [rank: 0] Seed set to 42
(RayTrainWorker pid=3011) [rank: 0] Seed set to 42
(RayTrainWorker pid=3011) [rank: 0] Seed set to 42
(RayTrainWorker p

Testing: |          | 0/? [00:00<?, ?it/s]
Testing DataLoader 2:  71%|███████▏  | 5/7 [00:00<00:00, 37.03it/s]


(RayTrainWorker pid=3011) 
(RayTrainWorker pid=3011)   | Name        | Type                        | Params | Mode 
(RayTrainWorker pid=3011) --------------------------------------------------------------------
(RayTrainWorker pid=3011) 0 | main_module | GRU4Rec                     | 82.8 K | train
(RayTrainWorker pid=3011) 1 | loss        | SequentialBCEWithLogitsLoss | 0      | train
(RayTrainWorker pid=3011) 2 | metrics     | RobustModuleDict            | 0      | train
(RayTrainWorker pid=3011) --------------------------------------------------------------------
(RayTrainWorker pid=3011) 82.8 K    Trainable params
(RayTrainWorker pid=3011) 0         Non-trainable params
(RayTrainWorker pid=3011) 82.8 K    Total params
(RayTrainWorker pid=3011) 0.331     Total estimated model params size (MB)
(RayTrainWorker pid=3011) 52        Modules in train mode
(RayTrainWorker pid=3011) 0         Modules in eval mode
(RayTrainWorker pid=3011) /Library/Frameworks/Python.framework/Versions/3.12/l

...

Testing DataLoader 0:  29%|██▊       | 2/7 [00:00<00:00, 21.71it/s]
(RayTrainWorker pid=3358) 
(RayTrainWorker pid=3365) Filtering by minimum number of users per item: 5 [repeated 3x across cluster]
(RayTrainWorker pid=3365) Filtering by minimum number of items per user: 5 [repeated 3x across cluster]
(RayTrainWorker pid=3365) Densifying index [repeated 3x across cluster]
Validation DataLoader 0:  57%|█████▋    | 4/7 [00:00<00:00, 35.89it/s] [repeated 40x across cluster]
(RayTrainWorker pid=3358) 
Testing DataLoader 1:  43%|████▎     | 3/7 [00:00<00:00, 37.55it/s]
(RayTrainWorker pid=3358) 
(RayTrainWorker pid=3358) 
(RayTrainWorker pid=3365) Splitting: leave_n_out [repeated 3x across cluster]
Testing DataLoader 2:  43%|████▎     | 3/7 [00:00<00:00, 45.87it/s]
(RayTrainWorker pid=3358) 
                                                                      
Epoch 1: 100%|██████████| 7/7 [00:00<00:00,  8.04it/s, v_num=0]
(RayTrainWorker pid=3365) Warning: RayTrainReportCallback not recog

(TorchTrainer pid=3338) Started distributed worker processes:  [repeated 2x across cluster]
(TorchTrainer pid=3338) - (node_id=fed15232356329568052a5376b724ec8d5728378a80c03a3ab3695fd, ip=127.0.0.1, pid=3365) world_rank=0, local_rank=0, node_rank=0 [repeated 2x across cluster]
(RayTrainWorker pid=3365) [rank: 0] Seed set to 42 [repeated 116x across cluster]
(RayTrainWorker pid=3365) GPU available: True (mps), used: False [repeated 3x across cluster]
(RayTrainWorker pid=3365) TPU available: False, using: 0 TPU cores [repeated 3x across cluster]
(RayTrainWorker pid=3365) HPU available: False, using: 0 HPUs [repeated 3x across cluster]
(RayTrainWorker pid=3365) /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`. [repeated 3x across cluster]
(RayTrainWorker pid=3365) /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packa

Epoch 0: 100%|██████████| 7/7 [00:00<00:00, 24.15it/s, v_num=0]
(RayTrainWorker pid=3365) 
Testing DataLoader 1:  71%|███████▏  | 5/7 [00:00<00:00, 33.50it/s] [repeated 2x across cluster]
(RayTrainWorker pid=3365) 
Testing DataLoader 1: 100%|██████████| 7/7 [00:00<00:00, 37.23it/s] [repeated 2x across cluster]
(RayTrainWorker pid=3365) 
Testing DataLoader 1:  29%|██▊       | 2/7 [00:00<00:00, 37.19it/s]
(RayTrainWorker pid=3365) 
(RayTrainWorker pid=3365) 
(RayTrainWorker pid=3365) ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓ [repeated 3x across cluster]
(RayTrainWorker pid=3365) ┃    Test metric    ┃   DataLoader 0    ┃   DataLoader 1    ┃   DataLoader 2   ┃ [repeated 3x across cluster]
(RayTrainWorker pid=3365) ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩ [repeated 3x across cluster]
(RayTrainWorker pid=3365) │     test_loss     │ 0.84604871273040… │ 0.84116393327713… │ 0.8333669900894… │ [repeated 3x across clus

(RayTrainWorker pid=3365) -------------------------------------------------------------------- [repeated 6x across cluster]
2024-12-24 13:25:10,949	INFO tune.py:1041 -- Total run time: 42.59 seconds (40.63 seconds for the tuning loop).
2024-12-24 13:25:10,978	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 88 trial(s):
- TorchTrainer_0519a_00012: FileNotFoundError('Could not fetch metrics for TorchTrainer_0519a_00012: both result.json and progress.csv were not found at /Users/federicosiciliano/ray_results/TorchTrainer_2024-12-24_13-24-26/TorchTrainer_0519a_00012_12_data_params_collator_params_lookback=5,model_optimizer_params_lr=0.0993,model_rec_model_dropout_input=_2024-12-24_13-24-28')
- TorchTrainer_0519a_00013: FileNotFoundError('Could not fetch metrics for TorchTrainer_0519a_00013: both result.json and progress.csv were not found at /Users/federicosiciliano/ray_results/TorchTrainer_2024-12-24_13-24-26/TorchTrainer_0519a_00013_13_data_params_collator_params_lookba

(RayTrainWorker pid=3358) /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(RayTrainWorker pid=3358)   warnings.warn('resource_tracker: There appear to be %d '


In [21]:
results

ResultGrid<[
  Result(
    metrics={'val_loss/dataloader_idx_0': 0.5177322626113892, 'val_Precision_@5/dataloader_idx_0': 0.06294643133878708, 'val_Precision_@10/dataloader_idx_0': 0.04821429029107094, 'val_Precision_@20/dataloader_idx_0': 0.03231026604771614, 'val_Recall_@5/dataloader_idx_0': 0.3147321343421936, 'val_Recall_@10/dataloader_idx_0': 0.4821428656578064, 'val_Recall_@20/dataloader_idx_0': 0.6462053656578064, 'val_MAP_@5/dataloader_idx_0': 0.0751078799366951, 'val_MAP_@10/dataloader_idx_0': 0.0640486404299736, 'val_MAP_@20/dataloader_idx_0': 0.05102028697729111, 'val_NDCG_@5/dataloader_idx_0': 0.20437857508659363, 'val_NDCG_@10/dataloader_idx_0': 0.258093923330307, 'val_NDCG_@20/dataloader_idx_0': 0.29930517077445984, 'val_MRR_@5/dataloader_idx_0': 0.16845238208770752, 'val_MRR_@10/dataloader_idx_0': 0.19038008153438568, 'val_MRR_@20/dataloader_idx_0': 0.20154595375061035, 'val_loss/dataloader_idx_1': 0.5141581296920776, 'val_Precision_@5/dataloader_idx_1': 0.05625000223517

In [23]:
results.get_best_result(metric="val_NDCG_@10/dataloader_idx_0", mode="max")

Result(
  metrics={'val_loss/dataloader_idx_0': 0.5177322626113892, 'val_Precision_@5/dataloader_idx_0': 0.06294643133878708, 'val_Precision_@10/dataloader_idx_0': 0.04821429029107094, 'val_Precision_@20/dataloader_idx_0': 0.03231026604771614, 'val_Recall_@5/dataloader_idx_0': 0.3147321343421936, 'val_Recall_@10/dataloader_idx_0': 0.4821428656578064, 'val_Recall_@20/dataloader_idx_0': 0.6462053656578064, 'val_MAP_@5/dataloader_idx_0': 0.0751078799366951, 'val_MAP_@10/dataloader_idx_0': 0.0640486404299736, 'val_MAP_@20/dataloader_idx_0': 0.05102028697729111, 'val_NDCG_@5/dataloader_idx_0': 0.20437857508659363, 'val_NDCG_@10/dataloader_idx_0': 0.258093923330307, 'val_NDCG_@20/dataloader_idx_0': 0.29930517077445984, 'val_MRR_@5/dataloader_idx_0': 0.16845238208770752, 'val_MRR_@10/dataloader_idx_0': 0.19038008153438568, 'val_MRR_@20/dataloader_idx_0': 0.20154595375061035, 'val_loss/dataloader_idx_1': 0.5141581296920776, 'val_Precision_@5/dataloader_idx_1': 0.05625000223517418, 'val_Precisi

In [20]:
#Problems with parallel execution:
# May generate the same experiment id (very unlikely)
### By default, there are N = 16^62 possible experiment ids
### L = already run experiments
### M = N - L possible experiment ids
### If generating K experiment ids concurrently, probability is sti
# TODO: reuse already run experiments, --> if using time budget, sleep for previous run-time (for fairness in comparison)